<a href="https://colab.research.google.com/github/laurarguezcorona/Calculadora-de-Bonos/blob/main/Calculo_de_VaR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

02.1 Cálculo del VaR con Python (colab)

Por favor, realice, en un archivo de Google colab, el calculo del valor en riesgo con los siguientes requisitos:

VaR a 98% de probabilidad de suceso a 1, 2, 3 ,4 y 5 días.
VaR a 95% de probabilidad de suceso a 1, 2, 3 ,4 y 5 días.
Calcule el VaR en pérdida monetaria, empleando variaciones porcentuales.
Calcule el VaR en pérdida monetaria, empleando P/L
Para evaluar el Var, utilice precios históricos diarios de los últimos 250 días, siendo la fecha de valuación el 9 de septiembre de 2024.

Proporcione la liga de su colab con privilegios de edición para fines de evaluación.

In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

In [2]:
# definir la fecha de valuación
valuation_date = "2024-09-09"

In [38]:
#Obtener los precios históricos de *MXX
# Obtener precios históricos de los últimos 250 días desde la fecha de valuación usaré 365 dias para garantizar que sean 250 dias efectivos.
ticker = "^MXX"
end_date = datetime.strptime(valuation_date, "2024-09-09")
start_date = end_date - timedelta(days=250)
data = yf.download(ticker, start=start_date, end=end_date)

[*********************100%***********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['^MXX']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 1899-04-26 00:00:00 -> 1900-01-01 00:00:00) (Yahoo error = "Data doesn\'t exist for startDate = -2230564980, endDate = -2208964980")')


In [39]:
#calcular los retornos diarios

data['Returns'] = data['Adj Close'].pct_change()
returns = data['Returns'].dropna()

ValueError: attempt to get argmax of an empty sequence

In [12]:
# Definir los niveles de confianza
confidence_levels = [0.98, 0.95]

In [17]:
# Definir horizontes (1 a 5 días)???

horizons = [1, 2, 3, 4, 5]

In [18]:
# Definir la Función para calcular el Valor en riesgo del activo *MXX
def calculate_var(returns, confidence_level, horizon):
    # VaR basado en percentil
    var_percentile = np.percentile(returns, (1 - confidence_level) * 100)
    # Ajustar el VaR al horizonte temporal
    var_horizon = var_percentile * np.sqrt(horizon)
    return var_horizon

In [21]:
# Crear un dataframe para almacenar los resultados de VaR porcentual
var_results = pd.DataFrame(columns=['Confidence Level', 'Horizon (days)', 'VaR (%)'])

In [ ]:
# Calcular el VaR para diferentes niveles de confianza y horizontes
for confidence_level in confidence_levels:
    for horizon in horizons:
        var = calculate_var(returns, confidence_level, horizon)
        var_results = var_results.append({
            'Confidence Level': confidence_level,
            'Horizon (days)': horizon,
            'VaR (%)': var * 100
        }, ignore_index=True)

In [26]:
# Mostrar resultados de VaR porcentual
print("VaR porcentual:")
print(var_results)

VaR porcentual:
Empty DataFrame
Columns: [Confidence Level, Horizon (days), VaR (%)]
Index: []


In [27]:
# Supongamos una inversión inicial de 100,000 USD para calcular el VaR en términos monetarios
investment = 100000

In [28]:
# Calcular el VaR en términos monetarios (variaciones porcentuales)
var_results['Monetary Loss (%)'] = var_results['VaR (%)'] / 100 * investment

In [29]:
# Mostrar resultados de pérdida monetaria usando variaciones porcentuales
print("\nVaR en términos monetarios usando variaciones porcentuales:")
print(var_results)


VaR en términos monetarios usando variaciones porcentuales:
Empty DataFrame
Columns: [Confidence Level, Horizon (days), VaR (%), Monetary Loss (%)]
Index: []


In [30]:
# Ahora vamos a calcular el VaR basado en P/L (ganancias y pérdidas)
# Calcular P/L diario (diferencias entre precios de cierre ajustado)
data['P/L'] = data['Adj Close'].diff()

In [31]:
# Crear un dataframe para almacenar los resultados de VaR en P/L
var_pl_results = pd.DataFrame(columns=['Confidence Level', 'Horizon (days)', 'VaR (P/L)'])

In [ ]:
# Calcular el VaR para diferentes niveles de confianza y horizontes usando P/L
for confidence_level in confidence_levels:
    for horizon in horizons:
        pl_var = calculate_var(data['P/L'].dropna(), confidence_level, horizon)
        var_pl_results = var_pl_results.append({
            'Confidence Level': confidence_level,
            'Horizon (days)': horizon,
            'VaR (P/L)': pl_var
        }, ignore_index=True)

In [33]:
# Calcular la pérdida monetaria basada en P/L
var_pl_results['Monetary Loss (P/L)'] = var_pl_results['VaR (P/L)']

In [34]:
# Mostrar los resultados de VaR en términos de P/L
print("\nVaR en términos monetarios usando P/L:")
print(var_pl_results)


VaR en términos monetarios usando P/L:
Empty DataFrame
Columns: [Confidence Level, Horizon (days), VaR (P/L), Monetary Loss (P/L)]
Index: []
